In [ ]:
!pip install spektral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import sklearn.neighbors as nn
import spektral

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import networkx as nx
import scipy
from scipy.sparse import coo_matrix

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/fer2013.csv')
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [ ]:
emotion_labels = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}
Labels = [2,3,4,6]
df = df[df.emotion.isin(Labels)]

In [ ]:
img_array = df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
img_array = np.stack(img_array, axis=0)
img_array.shape

(26385, 48, 48, 1)

In [ ]:
LE = LabelEncoder()
img_labels = LE.fit_transform(df['emotion'])
img_labels = np_utils.to_categorical(img_labels)
img_labels.shape

(26385, 4)

In [ ]:
LE_label_mapping = dict(zip(LE.classes_, LE.transform(LE.classes_)))
LE_label_mapping

{2: 0, 3: 1, 4: 2, 6: 3}

In [ ]:
# Create an empty graph
G = nx.Graph()

# Define the radius of connectivity
radius = 2

# Iterate over rows and columns of the pixel intensity array
for row in range(img_array.shape[1]):
    for col in range(img_array.shape[2]):
        node_id = row*48 + col
        pixel_intensity = img_array[0, row, col, :]

        G.add_node(node_id, pixel_intensity=pixel_intensity)

        # Connect to all nodes within the specified radius
        for i in range(max(0, row-radius), min(48, row+radius+1)):
            for j in range(max(0, col-radius), min(48, col+radius+1)):
                if i == row and j == col:
                    continue
                neighbor_id = i*48 + j
                G.add_edge(node_id, neighbor_id)


In [ ]:
print(nx.number_of_nodes(G))

2304


In [ ]:
print(nx.number_of_nodes(G))
print(nx.number_of_edges(G))

2304
26226


In [ ]:
A = nx.adjacency_matrix(G)

In [ ]:
!pip install --upgrade scipy networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
A.shape

(2304, 2304)

In [ ]:
img_array_feat = img_array.reshape(img_array.shape[0], -1, 1)

In [ ]:
img_array_norm = img_array_feat / 255.0

In [ ]:
img_array_norm.shape

(26385, 2304, 1)

In [ ]:
img_array_norm

array([[[0.90588236],
        [0.83137256],
        [0.6117647 ],
        ...,
        [0.34509805],
        [0.43137255],
        [0.59607846]],

       [[0.09411765],
        [0.1254902 ],
        [0.14117648],
        ...,
        [0.5568628 ],
        [0.56078434],
        [0.5568628 ]],

       [[0.01568628],
        [0.        ],
        [0.        ],
        ...,
        [0.11764706],
        [0.11372549],
        [0.11764706]],

       ...,

       [[0.69803923],
        [0.68235296],
        [0.6745098 ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.11764706],
        [0.10980392],
        [0.10980392],
        ...,
        [0.13725491],
        [0.11764706],
        [0.10980392]],

       [[0.07450981],
        [0.05098039],
        [0.05490196],
        ...,
        [0.7411765 ],
        [0.78039217],
        [0.7882353 ]]], dtype=float32)

In [ ]:
img_array.shape

(26385, 48, 48, 1)

In [ ]:
len(img_labels)

26385

In [ ]:
from spektral.layers import GCNConv
A = GCNConv.preprocess(A).astype('f4')
A = spektral.utils.normalized_laplacian(A, symmetric=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(img_array, img_labels,
                                                    shuffle=True, stratify=img_labels,
                                                    test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((21108, 48, 48, 1), (5277, 48, 48, 1), (21108, 4), (5277, 4))

In [ ]:
import tensorflow as tf
from spektral.utils import sp_matrix_to_sp_tensor
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Flatten, Dense, Lambda, Conv2D, MaxPooling2D, Reshape
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.models import Model, Sequential
from keras.regularizers import l2

In [ ]:
#X_in = Input(shape = (2304,1))
A_in = sp_matrix_to_sp_tensor(A)
l2_reg = 5e-4

"""
GCN1 = spektral.layers.GCNConv(channels = 512, activation='elu', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='GCN_1')([X_in,A_in])
B1 = BatchNormalization()(GCN1)
GCN2 = spektral.layers.GCNConv(channels = 256, activation='elu', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=, name='GCN2')([B1, A_in])
B2 = BatchNormalization()(GCN2)
# MP1 = spektral.layers.GlobalMaxPool()(B2)
D1 = Dropout(0.3)(B2)
GCN3 = spektral.layers.GCNConv(channels = 256 ,activation='elu', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=None, name='GCN3')([D1, A_in])
B3 = BatchNormalization()(GCN3)
GCN4 = spektral.layers.GCNConv(channels = 256 ,activation='elu', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=None, name='GCN4')([B3, A_in])
B4 = BatchNormalization()(GCN4)
#MP2 = spektral.layers.GlobalMaxPool()(B4)
D2 = Dropout(0.3)(B4)
GCN5 = spektral.layers.GCNConv(channels = 128 ,activation='elu', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=None, name='GCN5')([D2, A_in])
B5 = BatchNormalization()(GCN5)
GCN6 = spektral.layers.GCNConv(channels = 128 ,activation='elu', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=None, name='GCN6')([B5, A_in])
B6 = BatchNormalization()(GCN6)
#MP3 = spektral.layers.GlobalMaxPool()(B6)
D3 = Dropout(0.4)(B6)
flatten = Flatten()(D3)
DS1 = Dense(32,activation='elu',kernel_initializer='he_normal')(flatten)
B7 = BatchNormalization()(DS1)
D3 = Dropout(0.5)(B7)
output = Dense(4 , activation='softmax')(D3)
"""
"""
MP1 = spektral.layers.GlobalMaxPool()(X_in)
GCN1 = spektral.layers.GCNConv(channels = 32 ,activation='elu', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=None, name='conv2d_1')([MP1 ,A_in])
B1 = BatchNormalization()(GCN1)
D1 = Dropout(0.1)(B1)
GCN2 = spektral.layers.GCNConv(channels = 32 ,activation='elu', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=None, name='conv2d_2')([D1, A_in])
B2 = BatchNormalization()(GCN2)
MP2 = spektral.layers.GlobalMaxPool()(B2)
D2 = Dropout(0.1)(MP2)
flatten = Flatten()(D2)
DS1 = Dense(32, activation = "relu")(flatten)
out = Dense(4 , activation="softmax")(DS1)
"""

In [ ]:
img_width = X_train.shape[1]
img_height = X_train.shape[2]
img_depth = X_train.shape[3]
num_classes = y_train.shape[1]

In [ ]:
X_train = X_train / 255.
X_test = X_test / 255.

In [ ]:
A_in = sp_matrix_to_sp_tensor(A)
l2_reg = 5e-4

X_in = Input(shape = (48,48,1))

CN1 = Conv2D(filters=256,kernel_size=(5,5),input_shape=(img_width, img_height, img_depth), activation='elu',
                    padding='same',kernel_initializer='he_normal', name='conv2d_1')(X_in)
BN1 = BatchNormalization(name='batchnorm_1')(CN1)
#reshape2 = Reshape((48, 48, 256 ))(BN2)
CN2 = Conv2D(filters=128,kernel_size=(5,5),activation='elu',padding='same',kernel_initializer='he_normal',
                        name='conv2d_2')(BN1)
BN3 = BatchNormalization(name='batchnorm')(CN2)
#MP1 = MaxPooling2D(pool_size=(2,2), name='maxpool2d_1')(BN3)
D1 = Dropout(0.4, name='dropout_1')(BN3)
CN3 = Conv2D(filters=128,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal', name='conv2d_3')(D1)
BN4 = BatchNormalization(name='batchnorm_3')(CN3)
CN4 = Conv2D(filters=128,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',
                        name='conv2d_4')(BN4)
BN5 = BatchNormalization(name='batchnorm_4')(CN4)
#MP2 = MaxPooling2D(pool_size=(2,2), name='maxpool2d_2')(BN5)
D2 = Dropout(0.4, name='dropout_2')(BN5)
CN5 = Conv2D(filters=256,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',
                        name='conv2d_5')(D2)
BN6 = BatchNormalization(name='batchnorm_5')(CN5)
CN6 = Conv2D(filters=256,kernel_size=(3,3),activation='elu',padding='same',kernel_initializer='he_normal',
                        name='conv2d_6')(BN6)
BN7 = BatchNormalization(name='batchnorm_6')(CN6)
#MP3 = MaxPooling2D(pool_size=(2,2), name='maxpool2d_3')(BN7)
D3 = Dropout(0.5, name='dropout_3')(BN7)
# Add a Reshape layer
#reshaped = Reshape(2304,)(D3)
#GCN1 = spektral.layers.GCNConv(channels = 256, activation='elu', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='GCN_1')([D3,A_in])
#BN8 = BatchNormalization(name='batchnorm_6')(GCN1)
reshape1 = Reshape((2304, 256 ))(D3)
GCN1 = spektral.layers.GCNConv(channels = 256, activation='elu', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='GCN_1')([reshape1,A_in])
BN8 = BatchNormalization(name='batchnorm_2')(GCN1)
GCN2 = spektral.layers.GCNConv(channels = 256, activation='elu', use_bias=True, kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='GCN_2')([BN8,A_in])
BN9 = BatchNormalization(name='batchnorm_9')(GCN2)
MP3 = MaxPooling2D(pool_size=(2,2), name='maxpool2d_2')(BN9)
D1 = Dropout(0.5, name='dropout_3')(MP)
flatten = Flatten(name='flatten')(D3)
DS1 = Dense(128,activation='elu',kernel_initializer='he_normal',name='dense_1')(flatten)
BN9 = BatchNormalization(name='batchnorm_7')(DS1)
D4 = Dropout(0.6, name='dropout_4')(BN9)
output = Dense(4 , activation='softmax')(D4)

model = Model(inputs = [X_in], outputs = output)


In [ ]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 1)]       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 256)       6656      
                                                                 
 batchnorm_1 (BatchNormaliza  (None, 48, 48, 256)      1024      
 tion)                                                           
                                                                 
 reshape (Reshape)           (None, 2304, 256)         0         
                                                                 
 GCN_1 (GCNConv)             (None, 2304, 256)         65792     
                                                                 
 batchnorm_2 (BatchNormaliza  (None, 2304, 256)        1024      
 tion)                                                       

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy',min_delta=0.00005,patience=10,verbose=1,
    restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_accuracy',factor=0.5,patience=7,min_lr=1e-7,verbose=1)

callbacks = [early_stopping,lr_scheduler]

In [ ]:
batch_size=64
epochs=75
history = model.fit(x = X_train, y = y_train, batch_size = 32, epochs = epochs, validation_data = (X_test, y_test),
                    callbacks=callbacks)

Epoch 1/75
660/660 [==============================] - 85s 114ms/step - loss: 1.5918 - accuracy: 0.4519 - val_loss: 1.1785 - val_accuracy: 0.5657 - lr: 0.0010
Epoch 2/75
660/660 [==============================] - 73s 110ms/step - loss: 1.1237 - accuracy: 0.5596 - val_loss: 0.9982 - val_accuracy: 0.6182 - lr: 0.0010
Epoch 3/75
660/660 [==============================] - 78s 118ms/step - loss: 0.9926 - accuracy: 0.5978 - val_loss: 0.9243 - val_accuracy: 0.6394 - lr: 0.0010
Epoch 4/75
660/660 [==============================] - 78s 118ms/step - loss: 0.9349 - accuracy: 0.6156 - val_loss: 0.9083 - val_accuracy: 0.6301 - lr: 0.0010
Epoch 5/75
660/660 [==============================] - 78s 118ms/step - loss: 0.8829 - accuracy: 0.6347 - val_loss: 0.8536 - val_accuracy: 0.6506 - lr: 0.0010
Epoch 6/75
660/660 [==============================] - 79s 119ms/step - loss: 0.8522 - accuracy: 0.6521 - val_loss: 0.8376 - val_accuracy: 0.6602 - lr: 0.0010
Epoch 7/75
660/660 [==============================] 

In [ ]:
y_pred = model.predict(X_test)
y_true = y_test

165/165 [==============================] - 6s 33ms/step


In [ ]:
from sklearn.metrics import precision_score, recall_score
y_true = np.argmax(y_true, axis=1)
y_pred = np.argmax(y_pred, axis=1)

precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')

In [ ]:
print('Precision:', precision)
print('Recall:', recall)

Precision: 0.7055095567481405
Recall: 0.7029532876374842


In [ ]:
f1 = 2 * (precision * recall) / (precision + recall)

print('F1 score:', f1)

F1 score: 0.7042291024611304


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_true, y_pred)

In [ ]:
confusion_matrix

array([[ 635,   59,  212,  118],
       [  61, 1593,   57,   87],
       [ 166,   67,  733,  249],
       [  83,   99,  187,  871]])

In [ ]:
model.predict(X_test)

165/165 [==============================] - 17s 103ms/step


array([[0.19815241, 0.33802423, 0.23164697, 0.23217642],
       [0.1968421 , 0.33864844, 0.23119645, 0.23331304],
       [0.1794572 , 0.34674954, 0.2247494 , 0.24904382],
       ...,
       [0.19815241, 0.33802423, 0.23164697, 0.23217642],
       [0.19815241, 0.33802423, 0.23164697, 0.23217642],
       [0.19815241, 0.33802423, 0.23164697, 0.23217642]], dtype=float32)

In [ ]:
y_test

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [ ]:
X_test

array([[[0.62352943],
        [0.5921569 ],
        [0.61960787],
        ...,
        [0.4509804 ],
        [0.29803923],
        [0.38431373]],

       [[0.03137255],
        [0.02745098],
        [0.02352941],
        ...,
        [0.04705882],
        [0.03137255],
        [0.04313726]],

       [[0.19215687],
        [0.2509804 ],
        [0.3254902 ],
        ...,
        [0.48235294],
        [0.5137255 ],
        [0.5294118 ]],

       ...,

       [[0.6627451 ],
        [0.6627451 ],
        [0.6627451 ],
        ...,
        [0.14509805],
        [0.14509805],
        [0.16862746]],

       [[0.16862746],
        [0.16862746],
        [0.16862746],
        ...,
        [0.6117647 ],
        [0.5294118 ],
        [0.5019608 ]],

       [[0.39215687],
        [0.43137255],
        [0.45882353],
        ...,
        [0.58431375],
        [0.53333336],
        [0.5294118 ]]], dtype=float32)